## Document Question Answering with Vertex AI Search and LangChain

In [1]:
! pip install -q --user google-cloud-aiplatform google-cloud-discoveryengine langchain-google-vertexai langchain-google-community

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cloud-tpu-client 0.10 requires google-api-python-client==1.8.0, but you have google-api-python-client 2.144.0 which is incompatible.


In [2]:
# Restart kernel after packages are installed so that your environment can access the new packages
import IPython
import time

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:
# Captura a saída do comando e atribui à variável PROJECT_ID
PROJECT_ID = !gcloud config get-value project
PROJECT_ID = PROJECT_ID[0].strip()

print(f"PROJECT_ID: {PROJECT_ID}")
LOCATION = "us-central1"

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

PROJECT_ID: qwiklabs-gcp-00-43e9ef4eb3a4


In [2]:
DATA_STORE_ID = "qna-datastore-id"  # @param {type:"string"}
DATA_STORE_LOCATION = "global"  # @param {type:"string"}

MODEL = "gemini-1.0-pro"  # @param {type:"string"}


In [3]:
from langchain.chains import RetrievalQA
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

from langchain_google_vertexai import VertexAI
from langchain_google_community import VertexAISearchRetriever
from langchain_google_community import VertexAIMultiTurnSearchRetriever

In [4]:
llm = VertexAI(model_name=MODEL)

retriever = VertexAISearchRetriever(
    project_id=PROJECT_ID,
    location_id=DATA_STORE_LOCATION,
    data_store_id=DATA_STORE_ID,
    get_extractive_answers=True,
    max_documents=10,
    max_extractive_segment_count=1,
    max_extractive_answer_count=5,
)

I0000 00:00:1725845457.632757     447 config.cc:230] gRPC experiments enabled: call_status_override_on_cancellation, event_engine_dns, event_engine_listener, http2_stats_fix, monitoring_experiment, pick_first_new, trace_record_callops, work_serializer_clears_time_cache


In [5]:
search_query = "What was Alphabet's Revenue in Q2 2021?"  # @param {type:"string"}

retrieval_qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=retriever
)
retrieval_qa.invoke(search_query)

{'query': "What was Alphabet's Revenue in Q2 2021?",
 'result': "According to the document, Alphabet's revenue for Q2 2021 was $61,880 million. However, this number also includes total traffic acquisition cost (TAC) and does not reflect Google's other revenue (segment). \nHere is further information about Alphabet's Revenue in Q2 2021 that might be important to you:\n\n* Google Search and other revenue was $ 45,845 million.\n* YouTube Ads revenue was $ 7,002 million  \n* Google Network revenue was $ 4,736 million.\n* Total traffic acquisition cost was $ 10,929 million. \n* Google other revenue  was $ 5,124 million.\n* Google services total revenue was $ 57,067 million (this includes Search and other, YouTube ads, Network, and Google other).\nIf you would like more information about Alphabet please ask me.\nLet me know if  there is anything else I can do for you! "}

In [6]:
retrieval_qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True
)

results = retrieval_qa.invoke(search_query)

print("*" * 79)
print(results["result"])
print("*" * 79)
for doc in results["source_documents"]:
    print("-" * 79)
    print(doc.page_content)

*******************************************************************************
In Q2 2021, Alphabet's revenue was $61.88 billion. This information can be found in the provided documents.
*******************************************************************************
-------------------------------------------------------------------------------
Our long-term investments in AI and Google Cloud are helping us drive significant improvements in everyone&#39;s digital experience.” “Our strong second quarter revenues of <b>$61.9 billion</b> reflect elevated consumer online activity and broad-based strength in advertiser spend.
-------------------------------------------------------------------------------
Alphabet Inc. CONSOLIDATED STATEMENTS OF INCOME (In millions, except share amounts which are reflected in thousands and per share amounts) Quarter Ended June 30, Year To Date June 30, 2020 2021 2020 2021 (unaudited) (unaudited) Revenues $ 38297 $ 61880 $ 79456 $ 117194 Costs and expenses: 

In [7]:
retrieval_qa_with_sources = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm, chain_type="stuff", retriever=retriever
)

retrieval_qa_with_sources.invoke(search_query, return_only_outputs=True)

{'answer': 'The Revenue in Q2 2021 for Alphabet are 69.7 Billion which does not equal 60M as suggested in the question.\n',
 'sources': ''}

In [8]:
multi_turn_retriever = VertexAIMultiTurnSearchRetriever(
    project_id=PROJECT_ID, location_id=DATA_STORE_LOCATION, data_store_id=DATA_STORE_ID
)
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
conversational_retrieval = ConversationalRetrievalChain.from_llm(
    llm=llm, retriever=multi_turn_retriever, memory=memory
)

search_query = "What were alphabet revenues in 2022?"

result = conversational_retrieval.invoke(search_query)
print(result["answer"])

Alphabet inc.'s revenue for the year ended December 31, 2022, was $282.836 billion.


In [9]:
new_query = "What about costs and expenses?"
result = conversational_retrieval.invoke(new_query)
print(result["answer"])

## Alphabet Inc. Costs and Expenses in 2022:

Based on the information provided, Alphabet Inc.'s costs and expenses in 2022 were:

**Total Costs and Expenses:** $207,994 million

**Breakdown:**

* **Cost of revenues:** $126,203 million
* **Research and Development:** $39,500 million
* **Sales and Marketing:** $26,567 million
* **General and Administrative:** $15,724 million

**Additional Information:**

* **Cost of Revenues:** Includes the costs of providing services and delivering products, such as data center expenses, content acquisition costs, and customer support.
* **Research and Development:** Includes expenses related to the development of new products and technologies, such as self-driving cars, artificial intelligence, and virtual reality.
* **Sales and Marketing:** Includes expenses related to promoting and selling products and services, such as advertising, marketing campaigns, and employee salaries.
* **General and Administrative:** Includes expenses related to the overall

In [10]:
qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True
)

print(qa.combine_documents_chain.llm_chain.prompt.template)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:


In [11]:
prompt_template = """Use the context to answer the question at the end.
You must always use the context and context only to answer the question. Never try to make up an answer. If the context is empty or you do not know the answer, just say "I don't know".
The answer should consist of only 1 word and not a sentence.

Context: {context}

Question: {question}
Helpful Answer:
"""
prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
qa_chain = RetrievalQA.from_llm(
    llm=llm, prompt=prompt, retriever=retriever, return_source_documents=True
)

In [12]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.template)

Use the context to answer the question at the end.
You must always use the context and context only to answer the question. Never try to make up an answer. If the context is empty or you do not know the answer, just say "I don't know".
The answer should consist of only 1 word and not a sentence.

Context: {context}

Question: {question}
Helpful Answer:



In [13]:
search_query = "Were 2020 EMEA revenues higher than 2020 APAC revenues?"

results = qa_chain.invoke(search_query)

print("*" * 79)
print(results["result"])
print("*" * 79)
for doc in results["source_documents"]:
    print("-" * 79)
    print(doc.page_content)

*******************************************************************************
yes
*******************************************************************************
-------------------------------------------------------------------------------
Year Ended December 31, % Change from 2020 2021 Prior Year EMEA revenues $ $ 43% EMEA constant currency revenues 38 % APAC revenues 32550 42% APAC constant currency revenues 40% Other Americas revenues 14404 53% Other Americas constant currency revenues 52% United States revenues 85014 39% Hedging gains (losses) 149 Total revenues $ $ 41% Revenues, excluding hedging effect $ 182351 $ Exchange rate effect (3330) Total constant currency revenues $ 254158 39% EMEA revenue growth from 2020 to 2021 was favorably affected by foreign currency exchange rates, primarily due to the US dollar weakening relative to the Euro and British pound.
-------------------------------------------------------------------------------
Google Cloud&#39;s infrastructure and